# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Step 1: Dataset Exploration

Visualize the German Traffic Signs Dataset. This is open ended, some suggestions include: plotting traffic signs images, plotting the count of each sign, etc. Be creative!


The pickled data is a dictionary with 4 key/value pairs:

- features -> the images pixel values, (width, height, channels)
- labels -> the label of the traffic sign
- sizes -> the original width and height of the image, (width, height)
- coords -> coordinates of a bounding box around the sign in the image, (x1, y1, x2, y2)

In [ ]:
### Basic module imports and functions.

import cv2
import numpy as np


class Attributes(object):
    r'''Dynamic collection of named attributes.
    '''
    def __setattr__(self, name, value):
        object.__setattr__(self, name, value)

In [ ]:
### Basic dataset manipulation classes.

import pickle


class Dataset(object):
    def __init__(self, *args):
        if len(args) == 1:
            self.assign(args[0])
        elif len(args) == 2:
            self.title = args[0]
            self.load(args[1])
        elif len(args) == 3:
            self.title = args[0]
            self.X = args[1]
            self.y = args[2]
        else:
            raise Exception('Invalid argument list: %s' % str(args))

    def __len__(self):
        return len(self.X)
    
    def __str__(self):
        stats = self.stats()
        template = (
            '%s dataset\n'
            'Number of entries: %d\n'
            'Input shape: %s\n'
            'Number of classes: %d\n'
        )
        
        return template % (
            self.title,
            len(self),
            str(stats.inputs.shape),
            len(stats.classes)
        )

    def assign(self, dataset):
        self.title = dataset.title
        self.X = dataset.X
        self.y = dataset.y

    def load(self, path):
        with open(path, mode='rb') as data:
            dataset = pickle.load(data)
            self.X = dataset['features']
            self.y = dataset['labels']
    
    def stats(self):
        s = Attributes()
        s.inputs = Attributes()
        s.inputs.shape = self.X[0].shape
        
        y = self.y
        if len(y.shape) == 1:
            s.classes = set(y)
        else:
            s.classes = set(np.argmax(y_i) for y_i in y)
            
        return s
    
    
def Parameters():
    parameters = Attributes()
    parameters.inputs = Attributes()
    parameters.output = Attributes()
    parameters.patch = Attributes()
    parameters.batch = Attributes()
    return parameters

In [ ]:
### First we load the training and test datasets.

data = Attributes()
data.train = Dataset('Train', 'datasets/pickled/train.p')
data.tests = Dataset('Test', 'datasets/pickled/test.p')

parameters = Parameters()
parameters.inputs.side = data.train.X[0].shape[0]
parameters.output.classes = len(data.train.stats().classes)

### Do a basic data summary.

print(data.train)
print(data.tests)

In [ ]:
### Data exploration visualization.

import csv
from random import sample

from matplotlib import pyplot as plt
from matplotlib import cm


# Widen the page region used for plots
plt.rcParams['figure.figsize'] = (9.0, 4.0) # Otiginal: (6.0, 4.0)


class Displayer(object):
    def __init__(self, path):
        with open(path) as csv_file:
            csv_data = csv.reader(csv_file)
            next(csv_data) # Discard column title row
            self.labels = dict((int(row[0]), str(row[1])) for row in csv_data)

    def __call__(self, dataset, k=5):
        r'''Display `k` samples of each class from the given dataset.
        '''
        n = len(dataset)

        print('\n\n  %s' % ('-' * 80))
        print('  %s dataset (total %d entries)' % (dataset.title, n))
        print('  %s\n\n' % ('-' * 80))

        y = dataset.y
        labels = self.labels
        classes = dict((i, list()) for i in range(len(labels)))
        for i in range(n):
            y_i = y[i]
            if isinstance(y_i, np.ndarray):
                y_i = np.argmax(y_i)
            
            classes[y_i].append(i)

        for c in range(len(classes)):
            indexes = classes[c]
            print('  Class %d ("%s", total %d entries) samples:' % (c, labels[c], len(indexes)))
            s = sample(indexes, k)
            self.display_signs(dataset, s)

    def display_signs(self, dataset, indexes):
        r'''Display the indexed sign images and corresponding labels side by side.
        '''
        n = len(indexes)
        for i in range(n):
            plotter = plt.subplot2grid((1, n), (0, i))
            self.display_sign(plotter, dataset, indexes[i])

        plt.tight_layout()
        plt.show()

    def display_sign(self, plotter, dataset, i):
        r'''Display a sign image and corresponding numeric label.
        '''
        self.display_array(plotter, dataset.X[i])
        plotter.xaxis.set_visible(False)
        plotter.yaxis.set_visible(False)
        plotter.title.set_text(str(i))

    def display_array(self, plotter, x):
        channels = x.shape[2:]
        if channels == (3,) and x.dtype == np.uint8:
            plotter.imshow(x)
        elif channels == (1,):
            (i, j) = np.indices(x.shape[:2])
            plotter.matshow(x[i, j, 0], cmap=cm.gray)
        elif channels == ():
            plotter.matshow(x, cmap=cm.gray)
        else:
            raise Exception('Cannot display array of shape %s and type %s' % (str(x.shape), str(x.dtype)))

In [ ]:
# Display summary statistics and samples for the training and test datasets.

display = Displayer('datasets/signnames.csv')

display(data.train)
display(data.tests)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Your model can be derived from a deep feedforward net or a deep convolutional network.
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Data preprocessing facilities.

class Preprocessed(Dataset):
    def __init__(self, *args):
        Dataset.__init__(self, *args)
        X = self.X
        y = self.y
        stats = self.stats()
        image_shape = stats.inputs.shape
        shape = (len(self), image_shape[0], image_shape[1], 1)

        Xp = np.zeros(shape, dtype=np.float32)
        for (data, image) in zip(Xp, X):
            data.flat = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.float32).flat
            data -= np.mean(data)
            data /= np.std(data)

        n = len(stats.classes)
        yp = (np.arange(n) == y[:, None]).astype(np.float32)
        
        self.X = Xp
        self.y = yp

In [ ]:
# Preprocess training and test datasets.

data.train = Preprocessed(data.train)
data.tests = Preprocessed(data.tests)

parameters.inputs.depth = 1

display(data.train)
display(data.tests)

### Question 1 

_Describe the techniques used to preprocess the data._

**Answer:**

In [ ]:
### Separate a validation subset from the training dataset.

from sklearn.cross_validation import train_test_split


def split(dataset, rate=0.3):
    X = dataset.X
    y = dataset.y

    (X_train, X_valid, y_train, y_valid) = train_test_split(
        X,
        y,
        test_size=rate,
        random_state=832289
    )
    
    return (
        Dataset('Train', X_train, y_train),
        Dataset('Validate', X_valid, y_valid)
    )

In [ ]:
data.stats = data.train.stats()
(data.train, data.valid) = split(data.train)

print('Size of training dataset: %d entries' % len(data.train))
print('Size of validation dataset: %d entries' % len(data.valid))

### Question 2

_Describe how you set up the training, validation and testing data for your model. If you generated additional data, why?_

**Answer:**

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

import tensorflow as tf


def conn1d_layer(x, b_i, n, nonlinear=True):
    m = x.get_shape().as_list()[-1]
    W = tf.Variable(tf.truncated_normal([m, n], stddev=0.1))
    b = tf.Variable(tf.constant(b_i, shape=[n]))
    return tf.matmul(x, W) + b


def conv2d_layer(x, b_i, *shape):
    shape = list(shape)
    W = tf.Variable(tf.truncated_normal(shape, stddev=0.1)) 
    C = tf.nn.conv2d(x, W, [1, 2, 2, 1], padding='VALID')
    b = tf.Variable(tf.constant(b_i, shape=shape[-1:]))
    return tf.nn.relu(C + b)


def reshape2d(x):
    shape = x.get_shape()
    shape = [-1, shape[1:].num_elements()]
    return tf.reshape(x, shape)


class Network(object):
    def __init__(self, parameters):
        l_input = parameters.inputs.side
        d_input = parameters.inputs.depth
        l_patch = parameters.patch.side
        d_patch = parameters.patch.depth
        n_hidden = parameters.hidden_nodes
        n_classes = parameters.output.classes
        
        self.inputs = tf.placeholder(tf.float32, shape=(None, l_input, l_input, d_input))
        self.output = tf.placeholder(tf.float32)
        layer1 = conv2d_layer(self.inputs, 0.0, l_patch, l_patch, d_input, d_patch)
        layer2 = conv2d_layer(layer1, 1.0, l_patch, l_patch, d_patch, d_patch)
        layer3 = conn1d_layer(reshape2d(layer2), 1.0, n_hidden)
        layer4 = conn1d_layer(tf.nn.relu(layer3), 1.0, n_classes)
        self.graph = layer4

In [ ]:
parameters.patch.side = 5
parameters.patch.depth = 16
parameters.hidden_nodes = 64

network = Network(parameters)

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [ ]:
### Train your model here.
### Feel free to use as many code cells as needed.

from math import ceil, floor
from sys import maxsize

from tqdm import tqdm


def batches(dataset, l, counter=range):
    X = dataset.X
    y = dataset.y
    n = int(ceil(len(X) / l))
    for k in counter(n):
        a = k * l
        b = a + l
        yield (k, X[a:b], y[a:b])


def Loss(network):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(network.graph, network.output)
    return tf.reduce_mean(cross_entropy)


class Accuracy(object):
    def __init__(self, network, data, parameters):
        is_correct_prediction = tf.equal(tf.argmax(network.graph, 1), tf.argmax(network.output, 1))
        self.graph = tf.reduce_sum(tf.cast(is_correct_prediction, tf.float32))
        self.inputs = network.inputs
        self.output = network.output

        self.data = data
        self.batch_size = parameters.batch.size

    def __call__(self, session):
        inputs = self.inputs
        output = self.output
        graph = self.graph
        l = self.batch_size

        def accuracy(dataset):
            total = 0.0
            count = 0.0
            for (k, X_k, y_k) in batches(dataset, l):
                total += session.run(graph, feed_dict={inputs: X_k, output: y_k})
                count += len(X_k)

            return total / count

        return (
            accuracy(self.data.train),
            accuracy(self.data.valid)
        )

    
def plot_lines(title, x, *ys, **options):
    y_min = maxsize
    y_max = -maxsize

    plotter = plt.subplot(111)
    for (y, c, l) in ys:
        label = '%s (last: %.4f)' % (l, y[-1])
        plotter.plot(x, y, c, label=label)
        y_min = min(floor(np.min(y)), y_min)
        y_max = max(ceil(np.max(y)), y_max)

    plotter.set_title(title)
    plotter.set_xlim([x[0], x[-1]])
    plotter.set_ylim([y_min, y_max])
    plotter.legend(loc=options.get('loc', 1))
    #plotter.legend()
    plt.tight_layout()
    plt.show()


def train(network, data, parameters):
    batch_size = parameters.batch.size
    batch_step = parameters.batch.step
    learning_rate = parameters.learning_rate
    epochs = parameters.epochs

    inputs = network.inputs
    output = network.output
    loss = Loss(network)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)    
    accuracy = Accuracy(network, data, parameters)
    init = tf.initialize_all_variables()

    x_batch = []
    y_train = []
    y_valid = []
    y_loss = []
    with tf.Session() as session:
        session.run(init)
        for i in range(epochs):
            counter = lambda n: tqdm(range(n), desc='Epoch {:>2}/{}'.format(i + 1, epochs), unit='batches')
            for (k, X_k, y_k) in batches(data.train, batch_size, counter):
                (_, l) = session.run([optimizer, loss], feed_dict={inputs: X_k, output: y_k})

                if k % batch_step == 0:
                    (a_train, a_valid) = accuracy(session)
                    x_batch.append(len(x_batch) * batch_step)
                    y_train.append(a_train)
                    y_valid.append(a_valid)
                    y_loss.append(l)

            plot_lines('Loss', x_batch,
                (y_loss, 'g', 'Loss')
            )

            plot_lines('Accuracy', x_batch,
                (y_train, 'r', 'Training Accuracy'),
                (y_valid, 'b', 'Validation Accuracy'),
                loc=4
            )


def train_main():
    data = Attributes()
    data.train = Preprocessed('Train', 'datasets/pickled/train.p')
    data.tests = Preprocessed('Test', 'datasets/pickled/test.p')

    parameters = Parameters()
    parameters.inputs.side = data.train.X[0].shape[0]
    parameters.inputs.depth = 1
    parameters.output.classes = len(data.train.stats().classes)
    parameters.patch.side = 5
    parameters.patch.depth = 16
    parameters.hidden_nodes = 64
    parameters.batch.size = len(data.train) // 500
    parameters.batch.step = 10
    parameters.learning_rate = 0.05
    parameters.epochs = 5

    (data.train, data.valid) = split(data.train)
    
    network = Network(parameters)
    train(network, data, parameters)

In [ ]:
train_main()

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the dataset?_


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.